# 0. Data Loading

In [1]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define data path
data_path_sf = '../data/sofifa'

# Load data
try:
    players_15      = pd.read_csv(f"{data_path_sf}/players_15.csv", low_memory=False)
    players_16      = pd.read_csv(f"{data_path_sf}/players_16.csv", low_memory=False)
    players_17      = pd.read_csv(f"{data_path_sf}/players_17.csv", low_memory=False)
    players_18      = pd.read_csv(f"{data_path_sf}/players_18.csv", low_memory=False)
    players_19      = pd.read_csv(f"{data_path_sf}/players_19.csv", low_memory=False)
    players_20      = pd.read_csv(f"{data_path_sf}/players_20.csv", low_memory=False)
    players_21      = pd.read_csv(f"{data_path_sf}/players_21.csv", low_memory=False)
    players_22      = pd.read_csv(f"{data_path_sf}/players_22.csv", low_memory=False)
    players_23      = pd.read_csv(f"{data_path_sf}/players_23.csv", low_memory=False)

    male_teams      = pd.read_csv(f"{data_path_sf}/male_teams.csv")
    male_players    = pd.read_csv(f"{data_path_sf}/male_players.csv")
    male_coaches    = pd.read_csv(f"{data_path_sf}/male_coaches.csv")

    print("Files loaded successfully")

except FileNotFoundError as e:
    print(e)
    print(f"Error: One or more files at {data_path_sf} were not found.")
    exit()

Files loaded successfully


# 1. Data Exploration

## 1.1. Male Players

In [2]:
print(f"Dataset Shape: {male_players.shape}")
#print(f"\nColumn Data Types:\n{male_players.dtypes}")
#print(f"\nMissing Values:\n{male_players.isnull().sum()}")

Dataset Shape: (180021, 109)


## 1.2. Players 23

In [3]:
players_23 = players_23.rename(columns={"player_id": "sofifa_id"})
players_23_original = players_23.copy()
players_23 = players_23[players_23['fifa_version'] == 23]

print(f"Dataset Shape: {players_23.shape}")

Dataset Shape: (18533, 110)


# 2. Dataframe Creation

## 2.1. Male Players cleaning

In [4]:
# Convert fifa_version from float to int
male_players["fifa_version"] = male_players["fifa_version"].astype(int)

# Normalize player_url to full format
def normalize_url(url):
    if pd.isna(url):
        return url
    if url.startswith("https://sofifa.com/"):
        return url
    if url.startswith("/"):
        return "https://sofifa.com" + url
    return "https://sofifa.com/" + url.lstrip("/")

male_players["player_url"] = male_players["player_url"].apply(normalize_url)

## 2.2. Add URLs for player photos

In [5]:
# Create a dictionary automatically

df_by_version = {v: globals()[f"players_{v}"] for v in range(15, 24)}
df_by_version[24] = players_23   # FIFA 24 use players from 23 to get their photos

# Merge by version, by long_name + dob
frames = []

for version in male_players["fifa_version"].unique():

    subset = male_players[male_players["fifa_version"] == version].copy()
    src = df_by_version[version][["long_name", "dob", "player_face_url"]].drop_duplicates()

    merged = subset.merge(src, how="left", on=["long_name", "dob"])
    frames.append(merged)

male_players = pd.concat(frames, ignore_index=True)

# Fallback for FIFA 24 with short_name + dob
mask_missing = (male_players["fifa_version"] == 24) & (male_players["player_face_url"].isna())

players_23_short = players_23[["short_name", "dob", "player_face_url"]] \
                    .drop_duplicates(subset=["short_name", "dob"])

subset_short = male_players.loc[mask_missing, ["short_name", "dob"]].copy()
subset_short["idx"] = subset_short.index

merged_short = subset_short.merge(players_23_short, how="left", on=["short_name", "dob"])
merged_short = merged_short.set_index("idx")

male_players.loc[merged_short.index, "player_face_url"] = merged_short["player_face_url"]


In [6]:
# Reorder columns
cols = list(male_players.columns)
cols.remove("player_face_url")
pos = cols.index("player_url") + 1
cols.insert(pos, "player_face_url")

# Reorder dataframe
male_players = male_players[cols]

### 2.2.1 Volumetrías

In [7]:
vol_per_year = (
    male_players.groupby("fifa_version")["player_face_url"]
    .apply(lambda x: x.notna().sum())
    .reset_index(name="n_fotos")
)

total_per_year = male_players.groupby("fifa_version")["player_face_url"].size()
photo_per_year = vol_per_year.set_index("fifa_version")["n_fotos"]

porcentaje_por_anio = (photo_per_year / total_per_year * 100).round(2)

df_porcentages = pd.DataFrame({
    "fifa_version": total_per_year.index,
    "total_jugadores": total_per_year.values,
    "con_foto": photo_per_year.values,
    "porcentaje_con_foto": porcentaje_por_anio.values
})

print(f"Dataset Shape: {male_players.shape}")
df_porcentages

Dataset Shape: (180021, 110)


,fifa_version,total_jugadores,con_foto,porcentaje_con_foto
0,15,16182,15809,97.69
1,16,16706,15281,91.47
2,17,17596,17246,98.01
3,18,17954,17612,98.10
4,19,18086,17736,98.06
5,20,18483,18140,98.14
6,21,18892,18596,98.43
7,22,19239,18698,97.19
8,23,18533,17919,96.69
9,24,18350,12976,70.71


## 2.3. Añadir URL de las fotos de los clubes y nacionalidades

In [8]:
clubs_22 = players_22[
    ["club_team_id", "club_name", "club_logo_url", "club_flag_url"]
].drop_duplicates(subset=["club_team_id"], keep="first") \
 .reset_index(drop=True)

nations_22 = players_22[
    ["nationality_id", "nationality_name", "nation_logo_url", "nation_flag_url"]
].drop_duplicates(subset=["nationality_id"], keep="first") \
 .reset_index(drop=True)

print("Unique clubs:", len(clubs_22))
print("Unique nationalities:", len(nations_22))

Unique clubs: 702
Unique nationalities: 163


In [9]:
# Add club_logo_url and club_flag_url
male_players = male_players.merge(
    clubs_22[["club_team_id", "club_logo_url", "club_flag_url"]],
    on="club_team_id",
    how="left"
)

# Add nation_logo_url and nation_flag_url
male_players = male_players.merge(
    nations_22[["nationality_id", "nation_logo_url", "nation_flag_url"]],
    on="nationality_id",
    how="left"
)

# Sort club URLs behind club_team_id
cols = list(male_players.columns)
for col in ["club_flag_url", "club_logo_url"][::-1]:
    cols.insert(cols.index("club_team_id") + 1, cols.pop(cols.index(col)))
male_players = male_players[cols]

# Sort nation URLs behind nationality_id
cols = list(male_players.columns)
for col in ["nation_flag_url", "nation_logo_url"][::-1]:
    cols.insert(cols.index("nationality_id") + 1, cols.pop(cols.index(col)))
male_players = male_players[cols]

print(f"Dataset Shape: {male_players.shape}")

Dataset Shape: (180021, 114)


# 3. Cleaning the Final Dataset

In [10]:
print(f"Dataset Shape: {male_players.shape}")

Dataset Shape: (180021, 114)


In [11]:
# Remove unwanted columns
male_players = male_players.drop(columns=[
    "player_url", "fifa_update", "update_as_of", 
    "club_jersey_number", "league_level", "club_jersey_number",
    "club_position", "international_reputation",
    "real_face", "player_tags", "player_traits",
    "work_rate", "body_type", "nation_position",
    "nation_jersey_number",
    "physic", "skill_moves", "pace", "shooting",
    "passing", "dribbling", "defending"
], errors="ignore")

In [12]:
male_players.to_csv("../data/model_data/players.csv", index=False)